In [1]:
# make a data generator that yeild the features and labels
# we first need to cut the audio into frames
# then get the labels to fit those frames

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# cut audio into frames
import os
import csv

import numpy as np
import librosa
from librosa.display import waveshow, specshow
import torchaudio
import torch
from torchaudio import transforms as T 
from torchaudio import functional as F 
from matplotlib import pyplot as plt

from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, WIN_LENGTH, N_MELS, SAMPLE_RATE, N_FFT)
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils.files import get_file_name, list_file_paths_in_dir

from pitch_tracker.utils import dataset

/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [4]:
# melspectrogram_extractor = T.MelSpectrogram(
#     sample_rate=SAMPLE_RATE,
#     hop_length=HOP_LENGTH,
#     n_fft=N_FFT,
#     win_length=WIN_LENGTH,
#     pad_mode='constant',
#     center=True,
#     f_min=F_MIN,
#     f_max=None,
#     n_mels=N_MELS,
#     power=2.0,
#     norm="slaney",
# )



In [5]:
audio_path = '../medleydb/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
signal, sample_rate = load_audio_mono(audio_path, SAMPLE_RATE, keep_channel_dim=True)
# signal = torch.unsqueeze(signal,0)
# signal = torch.from_numpy(signal)
signal = signal.numpy()
mel_features = dataset.extract_melspectrogram_feature(signal,N_FFT, HOP_LENGTH, N_MELS, SAMPLE_RATE, backend='librosa')
print(mel_features.shape)

stft_feature = dataset.extract_stft_feature(signal, N_FFT, HOP_LENGTH)
print(stft_feature.shape)

(1, 88, 15258)
(1, 513, 15258)


In [6]:
csv_path = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/AClassicEducation_NightOwl.csv'
csv_folder = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/'
label_dict = dataset.create_label_dict(csv_folder)


In [7]:
# dataset.create_feature_label_generator(
#     audio_dir:str, 
#     label_dir:str, 
#     sample_rate:int, 
#     n_fft:int,
#     hop_length:int,
#     picking_frame_step:int,
#     picking_frame_size:int,
#     step_frame:int,
#     step_time:int,
#     dist_threshold:float,
#     empty_threshold:float
# )


In [14]:
dataset_paths = dataset.create_dataset_path_dict(csv_folder)

/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/multitrack.py:371: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(f_in)


In [15]:
dataset_paths

{'Phoenix_SeanCaughlinsTheScartaglen': ('/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/Phoenix_SeanCaughlinsTheScartaglen.csv',
  '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/data/Audio/Phoenix_SeanCaughlinsTheScartaglen/Phoenix_SeanCaughlinsTheScartaglen_MIX.wav'),
 'MusicDelta_FusionJazz': ('/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/MusicDelta_FusionJazz.csv',
  '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/data/Audio/MusicDelta_FusionJazz/MusicDelta_FusionJazz_MIX.wav'),
 'PurlingHiss_Lolita': ('/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/PurlingHiss_Lolita.csv',
  '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/data/Audio/PurlingHiss_Lolita/PurlingHiss_Lolita_MIX.wav'),
 'MusicDelta_Punk': ('/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label/2560/Melody2_midi/MusicDelta_Punk